Yahoo! Finance API with Python 2.7
=====

## Yahoo Finance dowload historical stock index data

-----

*September 3, 2017*<br>
*@bd_econ* 

This example retrieves the adjusted daily closing price for the Dow Jones Industrial Average, S&P 500 and Nasdaq 100. 

In [1]:
import requests
import re
import pandas as pd
from StringIO import StringIO
import time 

## Parameters/Settings

In [2]:
tickers = ['^DJI', '^GSPC', '^IXIC']
start = '1104537600'
base = 'https://finance.yahoo.com/quote/'
base2 = 'https://query1.finance.yahoo.com/v7/finance/download/'
end = int(time.time())
dates = '?period1={}&period2={}'.format(start, end)
param = '&interval=1d&filter=history&frequency=1d'

## Request Data

In [4]:
df = pd.DataFrame()

for t in tickers:
    url = '{}{}/history{}{}'.format(base, t, dates, param)
    s = requests.Session()
    r = s.get(url)
    regex = '"CrumbStore":{"crumb":"(.+?)"},'
    pattern = re.compile(regex)
    crumb = re.findall(pattern, r.content)[0]
    param2 = '{}{}&interval=1d&events=history&crumb={}/Q'.format(t, dates, crumb)
    url2 = '{}{}'.format(base2, param2)
    data = s.post(url2)
    csv = StringIO(data.content)
    df[t[1:]] = pd.read_table(csv, sep=',', 
                             parse_dates=['Date']).set_index('Date')['Adj Close'].round(2)
df.tail()

,DJI,GSPC,IXIC
Date,,,
2017-08-28,21808.40,2444.24,6283.02
2017-08-29,21865.37,2446.30,6301.89
2017-08-30,21892.43,2457.59,6368.31
2017-08-31,21948.10,2471.65,6428.66
2017-09-01,21987.56,2476.55,6435.33
